# Predicting used phone and tablet prices using regression

## Import Libraries

In [ ]:
import pandas as pd # Import Pandas for data manipulation using dataframes
import numpy as np # Import Numpy for data statistical analysis 
import matplotlib.pyplot as plt # Import matplotlib for data visualisation
import seaborn as sns # Statistical data visualization

## Import Data

In [ ]:
database = pd.read_csv('used_device_data.csv')

In [ ]:
np.random.seed(0) 

## Understanding the Data

In [ ]:
database.shape

In [ ]:
database.columns

In [ ]:
database.info()

In [ ]:
database.head(5)

## Missing Values

In [ ]:
#After understanding the database structure better, we check null values
missing_values_count = pd.DataFrame({'Null': database.isnull().sum()})
total = len(database)
percentage_null = round((missing_values_count['Null']/total)*100,1)
missing_values_count['Percentage'] = percentage_null
missing_values_count.sort_values(by= 'Null' , ascending = False)

In [ ]:
#We will be droping some columns that we won't be using
# database.drop(['lat', 'long', 'region_url', 'url', 'vin', 'county', 'id', 'image_url'], axis = 1, inplace = True)

In [ ]:
database.shape

In [ ]:
#Finally, we will check in case there are some duplicated rows.
database.duplicated().sum()

In [ ]:
#We drop the duplicates and keep one of each 
database = database.drop_duplicates(keep='first')